In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Sequential,load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image

In [2]:
train_dataset_path = "../dataset/train/"
test_dataset_path = "../dataset/test/"

In [3]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

In [4]:
train_generator = datagen.flow_from_directory(
    train_dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
)

test_generator = datagen.flow_from_directory(
    test_dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
)

Found 12000 images belonging to 4 classes.
Found 1805 images belonging to 4 classes.


In [5]:
class_names = ["Black Rot", "ESCA", "Healthy", "Leaf Blight"]

In [6]:
def create_model(optimizer):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(len(class_names), activation="softmax"))
    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [7]:
model1 = create_model(Adam(learning_rate=0.001))
model2 = create_model(Adam(learning_rate=0.0001))
model3 = create_model(SGD(learning_rate=0.01, momentum=0.9))

In [8]:
for i, model in enumerate([model1, model2, model3], start=1):
    print(f"Training model {i}")
    checkpoint = ModelCheckpoint(f"model{i}.h5", save_best_only=True)
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=10,
        validation_data=test_generator,
        validation_steps=test_generator.samples // test_generator.batch_size,
        callbacks=[checkpoint],
    )

Training model 1
Epoch 1/10
375/375 [==============================] - ETA: 0s - loss: 0.5890 - accuracy: 0.7782

c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


375/375 [==============================] - 733s 2s/step - loss: 0.5890 - accuracy: 0.7782 - val_loss: 0.1865 - val_accuracy: 0.9286
Epoch 2/10
375/375 [==============================] - 497s 1s/step - loss: 0.1951 - accuracy: 0.9327 - val_loss: 0.1844 - val_accuracy: 0.9369
Epoch 3/10
375/375 [==============================] - 482s 1s/step - loss: 0.1181 - accuracy: 0.9588 - val_loss: 0.1321 - val_accuracy: 0.9587
Epoch 4/10
375/375 [==============================] - 470s 1s/step - loss: 0.0585 - accuracy: 0.9806 - val_loss: 0.0927 - val_accuracy: 0.9715
Epoch 5/10
375/375 [==============================] - 490s 1s/step - loss: 0.0435 - accuracy: 0.9858 - val_loss: 0.1700 - val_accuracy: 0.9369
Epoch 6/10
375/375 [==============================] - 494s 1s/step - loss: 0.0396 - accuracy: 0.9863 - val_loss: 0.1126 - val_accuracy: 0.9721
Epoch 7/10
375/375 [==============================] - 560s 1s/step - loss: 0.0369 - accuracy: 0.9890 - val_loss: 0.1070 - val_accuracy: 0.9732
Epoch 8/10

In [6]:
testX, testY = next(test_generator)

In [7]:
models = [model1, model2, model3]

In [8]:
def ensemble_predictions(models, testX):
    results = [model.predict(testX) for model in models]
    results = np.array(results)
    summed = np.sum(results, axis=0)
    return np.argmax(summed, axis=1)


In [9]:
ensemble_pred = ensemble_predictions(models, testX)

1/1 [==============================] - 0s 317ms/step


In [10]:
accuracy = np.mean(ensemble_pred == np.argmax(testY, axis=1))
print('Ensemble Accuracy: %.3f' % (accuracy * 100))

Ensemble Accuracy: 100.000


In [11]:
img_path = "../black_rot_test.png"
img = image.load_img(img_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0

In [12]:
ensemble_pred = ensemble_predictions(models, img)

predicted_class = class_names[ensemble_pred[0]]
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 96ms/step
Predicted class: Black Rot
